In [2]:
import pandas as pd

In [4]:
completo = pd.read_csv("Datos_completos.csv")
completo

,index,Documento_Ingreso,Documento_Actual,Codigo,Apellido_1_Estudiante,Apellido_2_Estudiante,Nombres_Estudiante,Acceso,Subacceso,Correo_Institucional,...,Corte_3p,Nota_4to_Corte,Corte_4p,Nota_5to_Corte,Corte_5p,Nombre_completo,Correo,Grupo,usuario,Nota_final
0,0,529919,38547,920425,-,-,vnqjjarn,Termino condición doble programa,Bachillerato con Examen de Estado,vnqjjarn@urosario.edu.co,...,20,4.1,15,4.1,25,Norma Sarmiento,norma.sarmiento@urosario.edu.co,1,norma.sarmiento,4.10
1,1,259329,31151,499523,-,-,kqpzwhyu,Extranjero,Departamento de Admisiones,kqpzwhyu@urosario.edu.co,...,20,4.6,15,4.6,25,Norma Sarmiento,norma.sarmiento@urosario.edu.co,1,norma.sarmiento,4.52
2,2,945754,55708,143394,-,-,gjhgmojy,Reingreso por Fortalecimiento académico,GENERAL,gjhgmojy@urosario.edu.co,...,20,3.9,15,3.9,25,Norma Sarmiento,norma.sarmiento@urosario.edu.co,1,norma.sarmiento,3.80
3,3,648156,53483,898866,-,-,xhmffdtx,Reingreso por Fortalecimiento académico,Generación E,xhmffdtx@urosario.edu.co,...,20,3.3,15,3.3,25,Norma Sarmiento,norma.sarmiento@urosario.edu.co,1,norma.sarmiento,3.36
4,4,213957,81539,989640,-,-,mfrbreve,Nuevo al programa,GENERAL,mfrbreve@urosario.edu.co,...,20,3.1,15,3.1,25,Norma Sarmiento,norma.sarmiento@urosario.edu.co,1,norma.sarmiento,3.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,115,259329,31151,499523,-,-,kqpzwhyu,Extranjero,Departamento de Admisiones,kqpzwhyu@urosario.edu.co,...,20,3.4,15,3.4,25,Lazaro Ramirez,lazaro.ramirez@urosario.edu.co,1,lazaro.ramirez,3.62
116,116,945754,55708,143394,-,-,gjhgmojy,Reingreso por Fortalecimiento académico,GENERAL,gjhgmojy@urosario.edu.co,...,20,3.4,15,3.4,25,Lazaro Ramirez,lazaro.ramirez@urosario.edu.co,1,lazaro.ramirez,3.84
117,117,648156,53483,898866,-,-,xhmffdtx,Reingreso por Fortalecimiento académico,Generación E,xhmffdtx@urosario.edu.co,...,20,3.3,15,3.3,25,Lazaro Ramirez,lazaro.ramirez@urosario.edu.co,1,lazaro.ramirez,3.46
118,118,213957,81539,989640,-,-,mfrbreve,Nuevo al programa,GENERAL,mfrbreve@urosario.edu.co,...,20,2.9,15,2.9,25,Lazaro Ramirez,lazaro.ramirez@urosario.edu.co,1,lazaro.ramirez,2.84


%%latex
\usepackage{amsmath,amssymb,amsfonts}



### Tareas
1. Leer paper https://reader.elsevier.com/reader/sd/pii/S0360131516301634?token=DB86640804CC9FE6B7FB5B1D080396823D2094FD36E393FA33E1D62F8CCB331555272C9D164BCAFA28296E5B1C692630
2. Ver cada uno sus modelos teoricos (Escribirlos aca en latex en español), y despues aplicarlo a la base de datos completos
3. Reunion Lunes para explicarle al otro los modelos y la aplicación

Ruiz : K-Nearest, Decision Tree
Miguel : Regresion Logistica y Clasificador bayesiano

## Notas Paper Miguel

* Que el estudiante sepa del modelo predictivo podria considerarse malo, ya que él podria creer que sus notas futuras ya estan predichas por el modelo. Luego seria ideal que solo lo supiera el administrador
* Utilizaron tres notas de examenes como indicadores del performance durante el semestre. Asi como el GPA acumulativo fue tenido en cuenta y las notas en las materias pre-requisito de la materia.
* Las primeras notas del semestre tienden a ser muy buenas predictoras de como van a ser el rendimiento del estudiante de la materia
* Plantean que es mejor crear un modelo por cada materia que uno generalizado
* Recomiendan este libro para ver los metodos: "Principle of Data Mining,D.J. Hand, H. Mannila and P. Smyth MIT press, New York (2001)", Link : https://www.researchgate.net/publication/220688376_Principles_of_Data_Mining (Se ve como bueno)
* Calculan la precisión del modelo de los que pasaron con los que no, para encontrar un mejor modelo que prediga mejor a los malos. Si fueran unidos podria que un modelo diera menos errores en general pero que falle muchisimo en los malos estudiantes , lo cual seria un problema. Tambien calcularon un $F_{1.5}$ puntaje que le da mas peso a los estudiantes que fallaron que los que pasaron.
* Utilizaron el coeficiente de Pearson para saber que tan corelacionadas estaban las variables explicativas, si pasaba de 0,3 las escogian. (Podriamos hacer eso)
* Parece que es necesario acotar el numero de estudiantes de los datos de entrenamiento, para que el modelo no se descarrile. (Recordad base Juan F el numero de notas varia mucho).
* Falso negativo : No identificar estudiantes que se encontraban en riesgo.
* Termina siendo mejor haciendo un modelo combinado de otros modelos (Ensemble).

## Regresion Logistica

We the reccur to the function `count`, returns the count from the property, in this case Male and female. We will create this function. Be $z=(c_1,c_2)$ this vector which returns the count from every option there is possible, in this case Male("M") and Female("F").

The function to count for every group i is then :

$$
    f(D)_i = \sum_{i=1}^m k_i,k_i\in B_i
$$

$B_i=\{Male,Female\}$ are the possible groups and where $k_i$:

\begin{equation*}
    k_i= 
    \begin{cases}
    1  & \text{if $i\in B_i$ } \\
     0 & \text{if $i\notin B_i$  }
\end{cases}
\end{equation*}


## K-Nearest Neighbours

